In [1]:
# ensure relative imports are correct
import sys
sys.path.insert(0, '../..') # this notebook resides in a sub-sub-directory in the main package

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.QuantumMachine import QuantumMachine
from qm.qua import *

from instruments import LabBrick, MetaInstrument, Sa124
from instruments.quantum_machines.configuration import *

from measurement.routines import resonator_spectroscopy
from measurement.routines import resonator_power_level
from measurement.routines import qubit_spectroscopy
from measurement.routines import power_rabi
from measurement.routines import qubit_T1

In [3]:
# define a qubit
class Qubit(MetaInstrument):
    """
    Encapsulates a qubit.
    """
    def __init__(self, name: str, lo_freq: float):
        self.name = name
        self.lo_freq = lo_freq
    
    def _create_parameters(self):
        print("created dummy parameters for " + self.name)
    
    def _create_yaml_map(self):
        return {
            'lo_freq': self.lo_freq
        }

# instantiate the qubit
qubit = Qubit(name='qubit', lo_freq=4.36e9)

In [4]:
# define a resonator
class Resonator(MetaInstrument):
    """
    Encapsulates a resonator.
    """
    def __init__(self, name: str, lo_freq: float):
        self.name = name
        self.lo_freq = lo_freq
    
    def _create_parameters(self):
        print("created dummy parameters for " + self.name)
    
    def _create_yaml_map(self):
        return {
            'lo_freq': self.lo_freq
        }

# instantiate the resonator
rr = Resonator(name='rr', lo_freq=8.7571e9)

In [5]:
# start the qm
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

2021-04-13 20:06:45,368 - qm - INFO - Performing health check
2021-04-13 20:06:45,375 - qm - INFO - Health check passed


In [6]:
# We have tuned the IQ mixer according to the tutorial-2-2-SA124B-MixerTuning.ipynb document.
# These are key numbers obtained:
rr_dcOffsetI = 0.011558358245951837
rr_dcOffsetQ = -0.00947436793762317
rr_mixerGainCorrection = -0.15430981633780644
rr_mixerPhaseCorrection = 0.0212698142172309

q_dcOffsetI = -0.00647755186574999 
q_dcOffsetQ = -0.011624171790995147
q_mixerGainCorrection = -0.19547425898369505
q_mixerPhaseCorrection = 0.05154287262922243

# after this step we have tuned the IQ mixer.
qm.set_output_dc_offset_by_element("rr", "I", float(rr_dcOffsetI)) 
qm.set_output_dc_offset_by_element("rr", "Q", float(rr_dcOffsetQ))
qm.set_mixer_correction("mixer_rr", rr_IF, rr_LO, IQ_imbalance(rr_mixerGainCorrection, rr_mixerPhaseCorrection))
qm.set_output_dc_offset_by_element("qubit", "I", float(q_dcOffsetI)) 
qm.set_output_dc_offset_by_element("qubit", "Q", float(q_dcOffsetQ))
qm.set_mixer_correction("mixer_qubit", qubit_IF, qubit_LO, IQ_imbalance(rr_mixerGainCorrection, q_mixerPhaseCorrection))

In [7]:
# start the labbricks
lb_q = LabBrick(name='lb_q', serial_number=25331, element=qubit, power=15)
lb_rr = LabBrick(name='lb_rr', serial_number=25337, element=rr, power=13)

Trying to initialize lb_q
Connnected to LabBrick 25331
creating parameters...
Initializing device...
Successfully set frequency to 4.36E+09
Successfully set power to +15
Trying to initialize lb_rr
Connnected to LabBrick 25337
creating parameters...
Initializing device...
Successfully set frequency to 8.76E+09
Successfully set power to +13


In [94]:
with program() as script:
    play('CW', 'rr')
qm.execute(script)

2021-04-13 21:06:22,390 - qm - INFO - Flags: 
2021-04-13 21:06:22,390 - qm - INFO - Executing high level program


In [93]:
with program() as script:

    #I_st = declare_stream()
    #Q_st = declare_stream()
    
    #update_frequency("rr", rr_frequency)
    with infinite_loop_():
        play('gaussian', 'qubit', duration = int(145))
        #align('rr', 'qubit')
        wait(int(15e3), 'rr')
        #play('CW', 'qubit')
    #align('qubit', 'rr')
    #measure("long_readout" * amp(a), "rr", None, demod.full('long_integW1', I), demod.full('long_integW2', Q))
    #wait(WAIT_TIME, "rr")

    #with stream_processing():
    #    I_st.buffer(len(AMP_VEC)).average().save('I_mem')
    #    Q_st.buffer(len(AMP_VEC)).average().save('Q_mem')
    
qua_script = script
queued_job = qm.queue.add(qua_script)

2021-04-13 21:06:16,856 - qm - INFO - Flags: 
2021-04-13 21:06:16,857 - qm - INFO - Executing high level program


In [25]:
queued_job.cancel()

False

In [15]:
queued_job.halt

AttributeError: 'QmPendingJob' object has no attribute 'halt'

In [12]:
job = queued_job.wait_for_execution()